<h2 style="color: red;">Question 1</h2>


In [38]:
import pandas as pd
import numpy as np
import random
from datetime import datetime

def getTickerPrice(ticker: str, date: datetime) -> float:
        return random.uniform(1, 100)

def calculate_trade_performance(trades: pd.DataFrame) -> pd.Series:
    # Ensure 'Size' column is filled with 1 where missing
    trades['Size'] = trades['Size'].fillna(1)

    # Calculate returns for each trade
    trades['Return'] = 0.0
    trades['HoldingPeriod'] = 0
    
    # Handle long and short trades
    for idx, trade in trades.iterrows():
        price_at_close = getTickerPrice(trade['Symbol'], trade['Date'])
        if trade['Side'] == 'buy':
            trades.at[idx, 'Return'] = (price_at_close - trade['Price']) / trade['Price'] * trade['Size']
        elif trade['Side'] == 'sell':
            trades.at[idx, 'Return'] = (trade['Price'] - price_at_close) / trade['Price'] * trade['Size']
        
        # Calculate holding period (for simplicity, assume 1 day holding)
        trades.at[idx, 'HoldingPeriod'] = 1  
    
    # Total Return
    total_return = trades['Return'].sum()
    
    # Average Return per Trade
    avg_return_per_trade = trades['Return'].mean()
    
    # Win Rate
    win_rate = (trades['Return'] > 0).mean()
    
    # Profit Factor
    gross_profit = trades[trades['Return'] > 0]['Return'].sum()
    gross_loss = abs(trades[trades['Return'] < 0]['Return'].sum())
    profit_factor = gross_profit / gross_loss if gross_loss != 0 else np.nan
    
    # Maximum Drawdown
    cumulative_returns = trades['Return'].cumsum()
    max_drawdown = ((cumulative_returns.cummax() - cumulative_returns).max())
    
    # Average Holding Period
    avg_holding_period = trades['HoldingPeriod'].mean()
    
    # Sharpe Ratio
    avg_daily_return = trades['Return'].mean()
    daily_return_std = trades['Return'].std()
    sharpe_ratio = avg_daily_return / daily_return_std if daily_return_std != 0 else np.nan
    
    # Sortino Ratio
    downside_std = trades[trades['Return'] < 0]['Return'].std()
    sortino_ratio = avg_daily_return / downside_std if downside_std != 0 else np.nan
    
    # Volatility
    volatility = trades['Return'].std()
    
    # Exposure (assume all trades are held for one day, over the total days in the dataset)
    total_days = (trades['Date'].max() - trades['Date'].min()).days + 1
    exposure = len(trades) / total_days
    
    metrics = {
        'Total Return': total_return,
        'Average Return per Trade': avg_return_per_trade,
        'Win Rate': win_rate,
        'Profit Factor': profit_factor,
        'Maximum Drawdown': max_drawdown,
        'Average Holding Period': avg_holding_period,
        'Sharpe Ratio': sharpe_ratio,
        'Sortino Ratio': sortino_ratio,
        'Volatility': volatility,
        'Exposure': exposure
    }
    
    return pd.Series(metrics)
# Define data with required columns
data = {
    'Symbol': ['AAPL', 'GOOGL', 'TSLA'],
    'Date': [datetime(2023, 1, 1), datetime(2023, 1, 2), datetime(2023, 1, 3)],
    'Price': [150.0, 280.0, 700.0],
    'Side': ['buy', 'sell', 'buy'],
    'Size': [1, 2, None]  # None will be replaced with 1
}
df = pd.DataFrame(data)
print(calculate_trade_performance(df))


Total Return                0.149382
Average Return per Trade    0.049794
Win Rate                    0.333333
Profit Factor               1.084606
Maximum Drawdown            0.973794
Average Holding Period      1.000000
Sharpe Ratio                0.030777
Sortino Ratio               0.386980
Volatility                  1.617875
Exposure                    1.000000
dtype: float64


<h3 style="color: green;">Long Strategy: 
When the trade is a 'buy', the return is calculated based on the difference between the price at the time of selling and the price at the time of buying.
</h3><h3 style="color: green;">Short Strategy: 
When the trade is a 'sell', the return is calculated based on the difference between the price at the time of buying back the security and the price at the time of selling.
</h3>


<h2 style="color: red;">Question 2</h2>


In [ ]:
import pandas as pd

# Load the dataset
file_path = 'testData.csv' 
trades_df = pd.read_csv(file_path)

# Print the column names to inspect them
print(trades_df.columns)


In [ ]:
# Strip leading/trailing spaces from column names
trades_df.columns = trades_df.columns.str.strip()

# Print the column names again to confirm
print(trades_df.columns)


In [ ]:
# Convert the 'Date' column to datetime
if 'Date' in trades_df.columns:
    trades_df['Date'] = pd.to_datetime(trades_df['Date'])
else:
    print("Error: 'Date' column not found in the data.")

# Ensure 'Size' column is of float type and fill any missing values with 1
if 'Size' in trades_df.columns:
    trades_df['Size'] = trades_df['Size'].astype(float).fillna(1)
else:
    print("Error: 'Size' column not found in the data.")

# Ensure 'Price' column is of float type
if 'Price' in trades_df.columns:
    trades_df['Price'] = trades_df['Price'].astype(float)
else:
    print("Error: 'Price' column not found in the data.")

# Ensure 'Side' column contains only 'buy' or 'sell'
if 'Side' in trades_df.columns:
    trades_df['Side'] = trades_df['Side'].str.lower()
else:
    print("Error: 'Side' column not found in the data.")

# Display the preprocessed DataFrame
print(trades_df.head())


In [ ]:
import pandas as pd
import numpy as np
import random

# Load the dataset
file_path = 'testData.csv'
trades_df = pd.read_csv(file_path)

# Step 1: Rename columns
trades_df.rename(columns={
    'transactionDate': 'Date',
    'ticker': 'Symbol',
    'type': 'Side'
}, inplace=True)

# Step 2: Convert Date columns to datetime
trades_df['Date'] = pd.to_datetime(trades_df['Date'])

# Step 3: Map 'type' to 'Side'
trades_df['Side'] = trades_df['Side'].apply(lambda x: 'buy' if 'Purchase' in x else 'sell')

# Step 4: Calculate average amount from the amount range
def parse_amount(amount_str):
    amount_str = amount_str.replace('$', '').replace(',', '')
    low, high = amount_str.split(' - ')
    low = float(low)
    high = float(high)
    return (low + high) / 2

trades_df['Amount'] = trades_df['amount'].apply(parse_amount)

# Step 5: Generate random prices for each trade
np.random.seed(42)  # For reproducibility
trades_df['Price'] = np.random.uniform(100, 500, size=len(trades_df))

# Step 6: Add Size column (assuming each trade is 1 share)
trades_df['Size'] = 1

# Select only the necessary columns
trades_df = trades_df[['Date', 'Symbol', 'Side', 'Size', 'Price', 'Amount']]

# Display the transformed DataFrame
trades_df.head()


In [ ]:
# Auxiliary function to get ticker price 
def getTickerPrice(ticker: str, date: pd.Timestamp) -> float:
    return random.uniform(1, 100)

# Function to calculate trade performance metrics
def calculate_trade_performance(trades: pd.DataFrame) -> pd.Series:
    trades['Size'] = trades['Size'].fillna(1)
    trades['Return'] = 0.0
    trades['HoldingPeriod'] = 0
    
    for idx, trade in trades.iterrows():
        price_at_close = getTickerPrice(trade['Symbol'], trade['Date'])
        if trade['Side'] == 'buy':
            trades.at[idx, 'Return'] = (price_at_close - trade['Price']) / trade['Price'] * trade['Size']
        elif trade['Side'] == 'sell':
            trades.at[idx, 'Return'] = (trade['Price'] - price_at_close) / trade['Price'] * trade['Size']
        trades.at[idx, 'HoldingPeriod'] = 1
    
    total_return = trades['Return'].sum()
    avg_return_per_trade = trades['Return'].mean()
    win_rate = (trades['Return'] > 0).mean()
    gross_profit = trades[trades['Return'] > 0]['Return'].sum()
    gross_loss = abs(trades[trades['Return'] < 0]['Return'].sum())
    profit_factor = gross_profit / gross_loss if gross_loss != 0 else np.nan
    cumulative_returns = trades['Return'].cumsum()
    max_drawdown = ((cumulative_returns.cummax() - cumulative_returns).max())
    avg_holding_period = trades['HoldingPeriod'].mean()
    avg_daily_return = trades['Return'].mean()
    daily_return_std = trades['Return'].std()
    sharpe_ratio = avg_daily_return / daily_return_std if daily_return_std != 0 else np.nan
    downside_std = trades[trades['Return'] < 0]['Return'].std()
    sortino_ratio = avg_daily_return / downside_std if downside_std != 0 else np.nan
    volatility = trades['Return'].std()
    total_days = (trades['Date'].max() - trades['Date'].min()).days + 1
    exposure = len(trades) / total_days
    
    metrics = {
        'Total Return': total_return,
        'Average Return per Trade': avg_return_per_trade,
        'Win Rate': win_rate,
        'Profit Factor': profit_factor,
        'Maximum Drawdown': max_drawdown,
        'Average Holding Period': avg_holding_period,
        'Sharpe Ratio': sharpe_ratio,
        'Sortino Ratio': sortino_ratio,
        'Volatility': volatility,
        'Exposure': exposure
    }
    
    return pd.Series(metrics)

# Calculate trade performance metrics for the transformed DataFrame
trade_metrics = calculate_trade_performance(trades_df)

# Display the calculated metrics
print(trade_metrics)


<h2 style="color: red;">Using yfinance to download historic data and find retruns</h2>


In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import datetime as dt

# Load the dataset
file_path = 'testData.csv'
trades_df = pd.read_csv(file_path)

# Step 1: Rename columns
trades_df.rename(columns={
    'transactionDate': 'Date',
    'ticker': 'Symbol',
    'type': 'Side'
}, inplace=True)

# Step 2: Convert Date columns to datetime
trades_df['Date'] = pd.to_datetime(trades_df['Date'])

# Step 3: Map 'type' to 'Side'
trades_df['Side'] = trades_df['Side'].apply(lambda x: 'buy' if 'Purchase' in x else 'sell')

# Step 4: Calculate average amount from the amount range
def parse_amount(amount_str):
    amount_str = amount_str.replace('$', '').replace(',', '')
    low, high = amount_str.split(' - ')
    low = float(low)
    high = float(high)
    return (low + high) / 2

trades_df['Amount'] = trades_df['amount'].apply(parse_amount)

# Step 5: Download historical stock prices using yfinance
stocks = trades_df['Symbol'].unique().tolist()
start = trades_df['Date'].min() - pd.Timedelta(days=360)
end = trades_df['Date'].max() + pd.Timedelta(days=1)
ohlcv_data = {}

for ticker in stocks:
    ohlcv_data[ticker] = yf.download(ticker, start, end)

# Step 6: Fetch historical close prices for each trade
def get_actual_price(ticker, date):
    try:
        hist = ohlcv_data[ticker]
        if date in hist.index:
            return hist.loc[date, 'Adj Close']
        else:
            # Find the closest previous date
            closest_date = hist.index[hist.index <= date].max()
            return hist.loc[closest_date, 'Adj Close']
    except Exception as e:
        print(f"Error fetching price for {ticker} on {date}: {e}")
        return np.nan

trades_df['Price'] = trades_df.apply(lambda row: get_actual_price(row['Symbol'], row['Date']), axis=1)

# Drop rows where we couldn't fetch the actual price
trades_df.dropna(subset=['Price'], inplace=True)

# Step 7: Calculate the Size (number of shares) for each trade
trades_df['Size'] = trades_df['Amount'] / trades_df['Price']

# Step 8: Calculate trade performance metrics using actual prices
def calculate_trade_performance(trades: pd.DataFrame) -> pd.Series:
    trades = trades.sort_values('Date')
    trades['Return'] = 0.0
    
    portfolio = {}
    for idx, trade in trades.iterrows():
        symbol = trade['Symbol']
        if trade['Side'] == 'buy':
            if symbol not in portfolio:
                portfolio[symbol] = {'avg_price': trade['Price'], 'shares': trade['Size']}
            else:
                total_shares = portfolio[symbol]['shares'] + trade['Size']
                total_cost = (portfolio[symbol]['avg_price'] * portfolio[symbol]['shares']) + (trade['Price'] * trade['Size'])
                portfolio[symbol]['avg_price'] = total_cost / total_shares
                portfolio[symbol]['shares'] += trade['Size']
        elif trade['Side'] == 'sell':
            if symbol in portfolio and portfolio[symbol]['shares'] > 0:
                shares_sold = min(trade['Size'], portfolio[symbol]['shares'])
                trade_return = (trade['Price'] - portfolio[symbol]['avg_price']) / portfolio[symbol]['avg_price'] * shares_sold
                trades.at[idx, 'Return'] = trade_return
                portfolio[symbol]['shares'] -= shares_sold
                if portfolio[symbol]['shares'] == 0:
                    del portfolio[symbol]
    
    # Calculate metrics
    total_return = trades['Return'].sum()
    avg_return_per_trade = trades['Return'].mean()
    win_rate = (trades['Return'] > 0).mean()
    gross_profit = trades[trades['Return'] > 0]['Return'].sum()
    gross_loss = abs(trades[trades['Return'] < 0]['Return'].sum())
    profit_factor = gross_profit / gross_loss if gross_loss != 0 else np.inf
    cumulative_returns = (1 + trades['Return']).cumprod() - 1
    max_drawdown = (cumulative_returns.cummax() - cumulative_returns).max()
    
    # Calculate daily returns
    daily_returns = trades.groupby('Date')['Return'].sum()
    trading_days = (trades['Date'].max() - trades['Date'].min()).days
    annualized_return = ((1 + total_return) ** (252 / trading_days) - 1) if total_return > -1 else -1
    sharpe_ratio = np.sqrt(252) * daily_returns.mean() / daily_returns.std() if daily_returns.std() != 0 else 0
    sortino_ratio = np.sqrt(252) * daily_returns.mean() / daily_returns[daily_returns < 0].std() if len(daily_returns[daily_returns < 0]) > 0 else 0
    volatility = trades['Return'].std()
    
    metrics = {
        'Total Return': total_return,
        'Annualized Return': annualized_return,
        'Average Return per Trade': avg_return_per_trade,
        'Win Rate': win_rate,
        'Profit Factor': profit_factor,
        'Maximum Drawdown': max_drawdown,
        'Sharpe Ratio': sharpe_ratio,
        'Sortino Ratio': sortino_ratio,
        'Volatility': volatility
            }
    
    return pd.Series(metrics)

# Calculate trade performance metrics for the transformed DataFrame
trade_metrics = calculate_trade_performance(trades_df)

# Display the calculated metrics
print(trade_metrics)